In [ ]:
# run rabbitmq locally using docker
!docker rm -f rabbitmq || true 
!docker run --name rabbitmq -p 4369:4369 -p 5672:5672 -p 15672:15672 -p 25672:25672 --rm -d rabbitmq:3.7.2-management 


In [ ]:
import pika
import datetime
from random import *
from time import sleep

arguments = dict()
arguments['x-max-priority'] = 100 # this is how we set the available number of priorities

In [ ]:
# check rabbitmq is ready
# we will try to connect, with X attempts. 
attempt_to_connect = True
is_connected = False
max_number_of_connection_attempts = 10
connection_num = 1

while attempt_to_connect:
    try:
        cred = pika.PlainCredentials('guest', 'guest')
        connection = pika.BlockingConnection(pika.ConnectionParameters(host='127.0.0.1', credentials=cred))
        channel = connection.channel()
        attempt_to_connect = False
        is_connected = True
    except Exception, ex:
        if connection_num <= max_number_of_connection_attempts:
            print "{} of {} waiting to rabbitmq to be ready...".format(connection_num, max_number_of_connection_attempts)
            sleep(1)
            connection_num += 1
            attempt_to_connect = True            
        else:            
            attempt_to_connect = False
            
if is_connected:
    print "Connected to rabbitmq, get to mangement UI: http://localhost:15672"  
else:
    print "ERROR!!! Failed to connect to rabbitmq" 



In [ ]:
priority_queue = channel.queue_declare( queue="priority_queue", arguments=arguments) # create or get a q with priority
queue = channel.queue_declare( queue="queue") # create or get a simple q, no priorities

In [ ]:
print '# Messages in {}:{}'.format( 'priority_queue', priority_queue.method.message_count)
print '# Messages in {}:{}'.format( 'queue', queue.method.message_count)

In [ ]:
# add messages with random priorities
for message_id in range(10): 
    priority = randint(0,100) # set random priority
    properties = pika.spec.BasicProperties(priority=priority)
    message = str({"message":message_id,"x-message-priority":priority, "dt":str(datetime.datetime.now())})
    channel.basic_publish(exchange='', routing_key='priority_queue', body=message, properties=properties )
    channel.basic_publish(exchange='', routing_key='queue', body=message)
    


In [ ]:
print "* Messages in simple queue:"
msg = True
while msg:
    queue = channel.queue_declare( queue="queue")
    print '# Messages in queue {}'.format( queue.method.message_count)
    GetOk, props, msg = channel.basic_get(queue="queue", no_ack=True)
    if msg: print msg

print ""
print "* Messages in priority queue:"
msg = True
while msg:
    queue = channel.queue_declare( queue="priority_queue", arguments=arguments)
    print '# Messages in priority_queue {}'.format( queue.method.message_count)
    GetOk, props, msg = channel.basic_get(queue="priority_queue", no_ack=True)
    if msg: print msg